In [1]:
import pandas as pd
import numpy as np
import zipfile
from scipy import stats

In [2]:
df_desc = pd.read_csv('content/data_dictionary.csv')
df_desc

,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
...,...,...,...,...,...,...
76,Sleep Disturbance Scale,SDS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
77,Sleep Disturbance Scale,SDS-SDS_Total_Raw,Total Raw Score,int,NaN,NaN
78,Sleep Disturbance Scale,SDS-SDS_Total_T,Total T-Score,int,NaN,NaN
79,Internet Use,PreInt_EduHx-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN


In [3]:
df = pd.read_csv('content/train.csv')
df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [4]:
df.shape

(3960, 82)

# Before we begin...

Our data has categorical information, true numerical information, and categorical information stored as numbers

In [5]:
df_desc.head() # look at type column

,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN


In [6]:
df_desc['Type'].unique()

array(['str', 'float', 'categorical int', 'int'], dtype=object)

In [7]:
df_desc[df_desc['Field'] == 'sii'] # worthy to note that the description table doesn't have a row for the output sii

,Instrument,Field,Description,Type,Values,Value Labels


In [8]:
# categoricals = df.dtypes[df.dtypes == 'object'].index
categoricals = pd.Index(df_desc[df_desc['Type'] == 'str']['Field'])
categoricals # seasonal data + id

Index(['id', 'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season',
       'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_A-Season',
       'PAQ_C-Season', 'PCIAT-Season', 'SDS-Season', 'PreInt_EduHx-Season'],
      dtype='object', name='Field')

In [9]:
# allnumericals = df.dtypes[df.dtypes != 'object'].index
allnumericals = pd.Index(df_desc[df_desc['Type'] != 'str']['Field'])
allnumericals

Index(['Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
       'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
       'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone',
       'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
       'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03

In [10]:
# numcategoricals = allnumericals[(df[allnumericals].min() == 0) & (df[allnumericals].max() <= 5)]
numcategoricals = pd.Index(df_desc[df_desc['Type'] == 'categorical int']['Field']).append(pd.Index(['sii'],name='Field'))
numcategoricals

Index(['Basic_Demos-Sex', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND_Zone',
       'FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num',
       'BIA-BIA_Frame_num', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02',
       'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06',
       'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10',
       'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14',
       'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18',
       'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object', name='Field')

In [11]:
# numericals = allnumericals[~((df[allnumericals].min() == 0) & (df[allnumericals].max() <= 5))]
numericals = pd.Index(df_desc[(df_desc['Type'] == 'float') | (df_desc['Type'] == 'int')]['Field'])
numericals

Index(['Basic_Demos-Age', 'CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Height',
       'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_GSND',
       'FGC-FGC_GSD', 'FGC-FGC_PU', 'FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_TL',
       'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE',
       'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI',
       'BIA-BIA_Fat', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST',
       'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
       'PCIAT-PCIAT_Total', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T'],
      dtype='object', name='Field')

In [12]:
len(categoricals) + len(numcategoricals) + len(numericals), df.shape[1]

(82, 82)

# Cleaning up the Zeros

Some of the data is filled in with 0 when it really should be `NaN`

In [13]:
df.describe().loc['min'][df.describe().loc['min'] < 0].index

Index(['BIA-BIA_BMC', 'BIA-BIA_FMI', 'BIA-BIA_Fat'], dtype='object')

These columns have a negative minimum. Is this possible?

In [14]:
df_desc[df_desc['Field'].isin(df.describe().loc['min'][df.describe().loc['min'] < 0].index)]

,Instrument,Field,Description,Type,Values,Value Labels
35,Bio-electric Impedance Analysis,BIA-BIA_BMC,Bone Mineral Content,float,NaN,NaN
42,Bio-electric Impedance Analysis,BIA-BIA_FMI,Fat Mass Index,float,NaN,NaN
43,Bio-electric Impedance Analysis,BIA-BIA_Fat,Body Fat Percentage,float,NaN,NaN


You cannot have negative bone mineral content, negative fat mass index, or negative body fat percentage. We should replace the negative values with `NaN`.

In [15]:
replaceNegatives = df.describe().loc['min'][df.describe().loc['min'] < 0].index
replaceNegatives

Index(['BIA-BIA_BMC', 'BIA-BIA_FMI', 'BIA-BIA_Fat'], dtype='object')

In [16]:
categoricals.intersection(df.describe().loc['min'][df.describe().loc['min'] == 0].index)

Index([], dtype='object')

In [17]:
numericals.intersection(df.describe().loc['min'][df.describe().loc['min'] == 0].index)

Index(['Physical-BMI', 'Physical-Weight', 'Physical-Diastolic_BP',
       'Physical-Systolic_BP', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-FGC_CU', 'FGC-FGC_GSND', 'FGC-FGC_GSD', 'FGC-FGC_PU',
       'FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_TL', 'PCIAT-PCIAT_Total'],
      dtype='object')

These numerical columns have a minimum of 0. Is this possible?  
For most of them, they should not have values of 0. For example `Physical-Weight` of 0 does not mean anything, so we should replace 0s with `NaN`.  
However, the one exception is the column `PCIAT-PCIAT_Total`, which is the total score to the Parent-Child Internet Addiction Test, where it is possible to have a score of 0.

In [18]:
numcategoricals.intersection(df.describe().loc['min'][df.describe().loc['min'] == 0].index)

Index(['Basic_Demos-Sex', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU_Zone',
       'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone',
       'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
       'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',
       'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12',
       'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16',
       'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object')

These categorical columns have a minimum of 0. Is this possible?  
Most likely yes, since even though they are recorded as numbers, they are 'categorical'. For example, `Basic_Demos-Sex` is recorded as 0 and 1.

In [19]:
replaceZeros = numericals.intersection(df.describe().loc['min'][df.describe().loc['min'] == 0].index).drop('PCIAT-PCIAT_Total')
replaceZeros

Index(['Physical-BMI', 'Physical-Weight', 'Physical-Diastolic_BP',
       'Physical-Systolic_BP', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-FGC_CU', 'FGC-FGC_GSND', 'FGC-FGC_GSD', 'FGC-FGC_PU',
       'FGC-FGC_SRL', 'FGC-FGC_SRR', 'FGC-FGC_TL'],
      dtype='object')

In [20]:
df_desc[df_desc['Field'].isin(replaceZeros)]

,Instrument,Field,Description,Type,Values,Value Labels
7,Physical Measures,Physical-BMI,Body Mass Index (kg/m^2),float,NaN,NaN
9,Physical Measures,Physical-Weight,Weight (lbs),float,NaN,NaN
11,Physical Measures,Physical-Diastolic_BP,Diastolic BP (mmHg),int,NaN,NaN
13,Physical Measures,Physical-Systolic_BP,Systolic BP (mmHg),int,NaN,NaN
15,FitnessGram Vitals and Treadmill,Fitness_Endurance-Max_Stage,Maximum stage reached,int,NaN,NaN
16,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Mins,Exact time completed: Minutes,int,NaN,NaN
17,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Sec,Exact time completed: Seconds,int,NaN,NaN
19,FitnessGram Child,FGC-FGC_CU,Curl up total,int,NaN,NaN
21,FitnessGram Child,FGC-FGC_GSND,Grip Strength total (non-dominant),float,NaN,NaN
23,FitnessGram Child,FGC-FGC_GSD,Grip Strength total (dominant),float,NaN,NaN


A value of 0 is impossible for BMI, weight, blood pressure, or any fitness test that counts reps.

We should replace these 0 values with `NaN`.

In [21]:
df[replaceNegatives] = df[replaceNegatives].mask(df[replaceNegatives] <= 0)

In [22]:
(df[replaceNegatives] <= 0).sum().sum()

0

In [23]:
df[replaceZeros] = df[replaceZeros].mask(df[replaceZeros] <= 0)

In [24]:
(df[replaceZeros] <= 0).sum().sum()

0

In [25]:
df.isnull().sum().sum() / (df.shape[0] * df.shape[1])

0.4108770633160877

41% of the data is now null

In [317]:
# df[(df[df.dtypes.index[df.dtypes != 'object']] < 0).any(axis=1)]

In [318]:
# df[df.dtypes.index[df.dtypes != 'object']] < 0

In [319]:
# df.mask(df[df.dtypes.index[df.dtypes != 'object']] < 0, np.nan)
# df[df.dtypes.index[df.dtypes != 'object']].mask(df[df.dtypes.index[df.dtypes != 'object']] < 0, np.nan)

## Removing columns in which more than 80% of the rows are null

In [26]:
threshold = 0.8

In [27]:
(df.isnull().sum() > len(df)*threshold).sum()

5

There are 5 columns who have >80% missing values.

In [28]:
removeCols = df.columns[df.isnull().sum() > len(df)*threshold]

In [29]:
df_desc[df_desc['Field'].isin(removeCols)]

,Instrument,Field,Description,Type,Values,Value Labels
15,FitnessGram Vitals and Treadmill,Fitness_Endurance-Max_Stage,Maximum stage reached,int,NaN,NaN
16,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Mins,Exact time completed: Minutes,int,NaN,NaN
17,FitnessGram Vitals and Treadmill,Fitness_Endurance-Time_Sec,Exact time completed: Seconds,int,NaN,NaN
50,Physical Activity Questionnaire (Adolescents),PAQ_A-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
51,Physical Activity Questionnaire (Adolescents),PAQ_A-PAQ_A_Total,Activity Summary Score (Adolescents),float,NaN,NaN


In [30]:
"""
filter_null = df.isnull().sum().sort_values(ascending = True) <= (len(df) / 2)
filter_null = filter_null[filter_null == True]
list_of_columns_less_null = filter_null.index.to_list()
df_filter_null = df[list_of_columns_less_null]
df_filter_null.isna().sum().sum() / len(df_filter_null)
len(df_filter_null)
"""
df = df.drop(columns=removeCols)
df.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df.isnull().sum().sum() / (df.shape[0] * df.shape[1])

0.38279876688967596

38% of the data is now null.

In [32]:
(df.isnull().sum(axis=1) > df.shape[1]*threshold).sum()

756

756 rows of the data have more than 80% of the attributes missing! Consider removing those rows.  
On the other hand, we also have time series data as well for some of the rows, so we can wait and see later on if we really need to remove the rows that have a lot of missing values.

In [327]:
#numericals = numericals.drop(['Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Mins','Fitness_Endurance-Time_Sec','PAQ_A-PAQ_A_Total'])

In [328]:
#categoricals = categoricals.drop(['PAQ_A-Season'])

In [33]:
df_desc = df_desc[~(df_desc['Field'].isin(removeCols))]

In [34]:
numericals = pd.Index(df_desc[(df_desc['Type'] == 'float') | (df_desc['Type'] == 'int')]['Field'])
categoricals = pd.Index(df_desc[df_desc['Type'] == 'str']['Field'])
numcategoricals = pd.Index(df_desc[df_desc['Type'] == 'categorical int']['Field']).append(pd.Index(['sii'],name='Field'))

# Removing the outliers

In [35]:
df.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
count,3960.000000,3960.000000,2421.000000,3015.000000,3027.000000,3015.000000,898.000000,2953.000000,2967.000000,2953.000000,...,2728.000000,2725.000000,2728.000000,2730.000000,2733.000000,2736.000000,2609.000000,2606.000000,3301.000000,2736.000000
mean,10.433586,0.372727,65.454771,19.376812,55.946713,90.840060,27.278508,69.672536,81.597236,117.022689,...,1.452346,1.627890,1.613636,1.158974,0.943652,27.896199,41.088923,57.763622,1.060588,0.580409
std,3.574648,0.483591,22.341862,5.034191,7.473764,43.161374,5.567287,13.553022,13.665196,16.927687,...,1.495600,1.445622,1.529178,1.343661,1.185460,20.338853,10.427433,13.196091,1.094875,0.771122
min,5.000000,0.000000,25.000000,8.522436,33.000000,31.800000,18.000000,11.000000,27.000000,49.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,38.000000,0.000000,0.000000
25%,8.000000,0.000000,59.000000,15.890526,50.000000,58.200000,23.000000,61.000000,72.000000,107.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,33.000000,47.000000,0.000000,0.000000
50%,10.000000,0.000000,65.000000,17.950925,55.000000,77.800000,26.000000,68.000000,81.000000,114.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,26.000000,39.000000,55.000000,1.000000,0.000000
75%,13.000000,1.000000,75.000000,21.588631,62.000000,114.300000,30.000000,76.000000,90.500000,125.000000,...,2.000000,3.000000,2.000000,2.000000,1.000000,41.000000,46.000000,64.000000,2.000000,1.000000
max,22.000000,1.000000,999.000000,59.132048,78.500000,315.000000,50.000000,179.000000,138.000000,203.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,93.000000,96.000000,100.000000,3.000000,3.000000


In [36]:
df[(np.abs(stats.zscore(df[numericals], nan_policy='omit')) >= 3).any(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.00,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
51,02cebf33,Spring,12,1,Spring,77.0,Summer,19.751301,62.00,108.0,...,3.0,3.0,3.0,46.0,Summer,82.0,100.0,Spring,1.0,1.0
59,0380e709,Summer,17,0,NaN,NaN,Summer,24.446886,68.00,160.8,...,4.0,2.0,2.0,56.0,NaN,NaN,NaN,Summer,NaN,2.0
87,04df895a,Winter,17,0,NaN,NaN,Winter,26.630764,77.00,224.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,2.0,NaN
193,0acac70c,Spring,18,0,Fall,65.0,Summer,21.485616,70.15,150.4,...,0.0,3.0,4.0,34.0,Summer,33.0,47.0,Spring,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3876,f9ca9ff6,Winter,11,1,NaN,NaN,Spring,16.319643,56.00,72.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,1.0,NaN
3903,fb9b060f,Summer,16,1,Fall,65.0,Summer,21.266964,67.00,135.8,...,3.0,2.0,1.0,52.0,Summer,68.0,93.0,Summer,2.0,2.0
3922,fcd9a6bb,Spring,6,0,Summer,69.0,Spring,16.195145,44.00,44.6,...,1.0,0.0,0.0,14.0,Spring,38.0,54.0,Spring,0.0,0.0
3946,feb53635,Winter,14,0,Summer,70.0,Spring,19.036779,69.50,130.8,...,4.0,4.0,4.0,77.0,Spring,51.0,71.0,Winter,3.0,2.0


In [333]:
# df[((np.abs(stats.zscore(df[numericals], nan_policy='omit')) < 3) | (np.abs(stats.zscore(df[numericals], nan_policy='omit')).isnull())).all(axis=1)]

If we track outliers based on a $|zscore| >= 3$ metric, then that takes out 317 rows of the data!

In [37]:
317 / df.shape[0]

0.08005050505050505

8% outliers based on zscore

But, the data is mostly skewed, so perhaps we can try something other than zscore for outlier detection.  
How about based on IQR?

In [38]:
Q1 = df[numericals].quantile(0.25)
Q3 = df[numericals].quantile(0.75)
IQR = Q3 - Q1

In [39]:
df[((df[numericals] < (Q1 - 1.5 * IQR)) | (df[numericals] > (Q3 + 1.5 * IQR))).any(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.50,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.00,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
21,00f574e9,Summer,16,0,NaN,NaN,Summer,29.285856,71.00,210.0,...,2.0,1.0,0.0,30.0,NaN,NaN,NaN,Summer,3.0,0.0
22,01085eb3,Fall,12,0,Winter,58.0,Fall,34.187282,60.50,178.0,...,1.0,1.0,0.0,30.0,Fall,53.0,73.0,Fall,0.0,0.0
32,01d24f60,Fall,16,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Winter,67.0,91.0,Fall,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3935,fdd9799f,Spring,15,0,Summer,48.0,Summer,29.404637,66.00,182.2,...,2.0,4.0,2.0,45.0,Summer,40.0,56.0,Spring,3.0,1.0
3938,fe102a25,Summer,5,0,Winter,61.0,Summer,18.278557,43.50,49.2,...,3.0,0.0,0.0,34.0,NaN,NaN,NaN,Summer,3.0,1.0
3945,fe9c71d8,Winter,9,0,Spring,80.0,Winter,15.891137,56.75,72.8,...,1.0,1.0,1.0,29.0,Winter,33.0,47.0,Winter,0.0,0.0
3946,feb53635,Winter,14,0,Summer,70.0,Spring,19.036779,69.50,130.8,...,4.0,4.0,4.0,77.0,Spring,51.0,71.0,Winter,3.0,2.0


In [40]:
784 / df.shape[0]

0.19797979797979798

20% outliers according to using the IQR!  
That's a lot of the data considered as outliers, perhaps we should widen the outlier fence boundaries

In [41]:
df[((df[numericals] < (Q1 - 2.5 * IQR)) | (df[numericals] > (Q3 + 2.5 * IQR))).any(axis=1)]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
6,0038ba98,Fall,10,0,NaN,NaN,Fall,19.660760,55.0,84.6,...,4.0,1.0,0.0,20.0,Winter,27.0,40.0,Fall,3.0,0.0
22,01085eb3,Fall,12,0,Winter,58.0,Fall,34.187282,60.5,178.0,...,1.0,1.0,0.0,30.0,Fall,53.0,73.0,Fall,0.0,0.0
51,02cebf33,Spring,12,1,Spring,77.0,Summer,19.751301,62.0,108.0,...,3.0,3.0,3.0,46.0,Summer,82.0,100.0,Spring,1.0,1.0
59,0380e709,Summer,17,0,NaN,NaN,Summer,24.446886,68.0,160.8,...,4.0,2.0,2.0,56.0,NaN,NaN,NaN,Summer,NaN,2.0
144,0844aee7,Summer,8,0,Fall,61.0,Fall,14.706858,48.0,48.2,...,1.0,1.0,1.0,32.0,Fall,32.0,46.0,Summer,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3844,f7fc97cf,Spring,14,0,Spring,75.0,Spring,23.752311,69.5,163.2,...,1.0,1.0,2.0,23.0,Spring,35.0,50.0,Spring,2.0,0.0
3863,f8ff0bc8,Fall,18,0,Spring,62.0,Winter,26.429967,71.5,192.2,...,1.0,1.0,2.0,24.0,Winter,42.0,59.0,Fall,2.0,0.0
3922,fcd9a6bb,Spring,6,0,Summer,69.0,Spring,16.195145,44.0,44.6,...,1.0,0.0,0.0,14.0,Spring,38.0,54.0,Spring,0.0,0.0
3946,feb53635,Winter,14,0,Summer,70.0,Spring,19.036779,69.5,130.8,...,4.0,4.0,4.0,77.0,Spring,51.0,71.0,Winter,3.0,2.0


In [42]:
274 / df.shape[0]

0.0691919191919192

Let's remove this 7% of the data that is extreme

In [43]:
df = df[~(((df[numericals] < (Q1 - 1.5 * IQR)) | (df[numericals] > (Q3 + 1.5 * IQR))).any(axis=1))]
df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


## Impute NaN values with random values from the same column

In [44]:
df.isnull().sum().sum()

102547

Let's impute with random values from the same column

In [45]:
df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,NaN,NaN,Spring,18.764678,53.5,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter,0.0,NaN
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


In [46]:
for col in df.columns:
    choices = df[col].dropna().values
    
    nan_indices = df[col].isnull()
    df.loc[nan_indices, col] = np.random.choice(choices, size=nan_indices.sum())

In [47]:
df.isnull().sum().sum()

0

In [48]:
df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,Spring,34.0,55.0,Fall,3.0,2.0
1,000fd460,Summer,9,0,Spring,60.0,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,60.0,Summer,14.865436,52.0,101.4,...,1.0,0.0,1.0,20.0,Spring,36.0,58.0,Spring,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3956,ffa9794a,Winter,10,0,Summer,60.0,Spring,18.764678,53.5,76.4,...,0.0,0.0,1.0,65.0,Spring,43.0,68.0,Winter,0.0,0.0
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0
3958,ffed1dd5,Spring,13,0,Spring,70.0,Winter,12.235895,70.7,87.0,...,1.0,1.0,1.0,19.0,Spring,33.0,47.0,Spring,1.0,0.0


# One Hot encoding

In [49]:
categoricals

Index(['id', 'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season',
       'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_C-Season',
       'PCIAT-Season', 'SDS-Season', 'PreInt_EduHx-Season'],
      dtype='object', name='Field')

Excluding the id, everything else is season data, so let's one hot encode them

In [50]:
categoricals = categoricals.drop('id')

In [51]:
from sklearn.preprocessing import OneHotEncoder

In [52]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df[categoricals])

one_hot_df = pd.DataFrame(one_hot_encoded,
                          columns=encoder.get_feature_names_out(categoricals))

print(one_hot_df.shape)
print(df.shape)

df.reset_index(drop=True, inplace=True)
one_hot_df.reset_index(drop=True, inplace=True)
df = pd.concat([df.drop(categoricals, axis=1), one_hot_df], axis=1)

print(df.shape)

df

(3176, 40)
(3176, 77)
(3176, 107)


,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
0,00008ff9,5,0,51.0,16.877316,46.0,50.8,22.0,67.0,83.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,000fd460,9,0,60.0,14.035590,48.0,46.0,22.0,75.0,70.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,00105258,10,1,71.0,16.648696,56.5,75.6,22.0,65.0,94.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,00115b9f,9,0,71.0,18.292347,56.0,81.6,26.0,60.0,97.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0016bb22,18,1,60.0,14.865436,52.0,101.4,21.0,53.0,113.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3171,ff8a2de4,13,0,60.0,16.362460,59.5,82.4,26.0,71.0,70.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3172,ffa9794a,10,0,60.0,18.764678,53.5,76.4,27.0,60.0,78.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3173,ffcd4dbd,11,0,68.0,21.441500,60.0,109.8,30.0,79.0,99.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3174,ffed1dd5,13,0,70.0,12.235895,70.7,87.0,27.0,59.0,61.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


Let's check it

In [53]:
df.iloc[:,-4:]

,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0
...,...,...,...,...
3171,1.0,0.0,0.0,0.0
3172,0.0,0.0,0.0,1.0
3173,1.0,0.0,0.0,0.0
3174,0.0,1.0,0.0,0.0


# Normalization

We'll use min max normalization since the data is skewed.

We will first split the data into train and test  
Then we'll scale the numerical columns afterward

In [54]:
from sklearn.preprocessing import MinMaxScaler

In [55]:
# shuffle the data!
df = df.sample(frac=1)
df

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
2125,a8722cde,17,0,60.0,17.995666,63.00,101.6,28.0,68.0,68.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
835,3e377cb7,10,0,80.0,23.122121,58.25,111.6,23.0,57.0,94.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1173,59ea397f,7,1,68.0,18.320842,47.50,58.8,25.0,60.0,75.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2746,dd4b8b52,13,1,70.0,20.154955,51.00,69.4,30.0,67.0,77.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2715,db49ac7c,12,0,78.0,16.091260,58.00,77.0,23.0,67.0,79.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,5c2c136d,10,0,55.0,21.147561,53.00,84.5,30.0,55.0,77.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
605,2c512f2f,9,1,70.0,20.415120,50.00,72.6,21.0,65.0,88.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2200,aea63a78,10,1,80.0,20.829630,60.00,146.2,24.0,72.0,74.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
147,09eeaba6,7,1,90.0,17.568743,53.00,70.2,24.0,72.0,80.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [56]:
dfTest = df.iloc[:df.shape[0] // 10]

In [57]:
dfTrain = df.iloc[df.shape[0] // 10:]

In [58]:
dfTrain[numericals] = dfTrain[numericals].astype('float64')
dfTest[numericals] = dfTest[numericals].astype('float64')

C:\Users\kerry\AppData\Local\Temp\ipykernel_30612\2330913363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrain[numericals] = dfTrain[numericals].astype('float64')
C:\Users\kerry\AppData\Local\Temp\ipykernel_30612\2330913363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTest[numericals] = dfTest[numericals].astype('float64')


In [59]:
scaler = MinMaxScaler()
# df_normalized = pd.DataFrame(scaler.fit_transform(df[numericals]), columns=numericals)
dfTrain.loc[:, numericals] = scaler.fit_transform(dfTrain[numericals])

dfTest.loc[:, numericals] = scaler.transform(dfTest[numericals])

In [60]:
dfTrain

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
361,18f6e6db,0.066667,0,0.500000,0.356925,0.433673,0.152778,0.571429,0.362069,0.890411,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2353,bcf577e2,0.133333,1,0.583333,0.448477,0.267857,0.152778,0.190476,0.965517,0.424658,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1820,8f102e6e,0.200000,1,0.600000,0.720436,0.757653,0.486111,0.333333,0.689655,0.575342,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
191,0c8efce0,0.133333,0,0.416667,0.316338,0.357143,0.143939,0.095238,0.396552,0.232877,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2856,e6276aaa,0.200000,0,0.266667,0.422493,0.446429,0.473485,0.190476,0.379310,0.150685,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,5c2c136d,0.333333,0,0.333333,0.584652,0.433673,0.332702,0.523810,0.275862,0.438356,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
605,2c512f2f,0.266667,1,0.583333,0.550734,0.357143,0.257576,0.095238,0.448276,0.589041,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2200,aea63a78,0.333333,1,0.750000,0.569929,0.612245,0.722222,0.238095,0.568966,0.397260,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
147,09eeaba6,0.133333,1,0.916667,0.418922,0.433673,0.242424,0.238095,0.568966,0.479452,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [61]:
dfTest

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
2125,a8722cde,0.800000,0,0.416667,0.438692,0.688776,0.440657,0.428571,0.500000,0.315068,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
835,3e377cb7,0.333333,0,0.750000,0.676091,0.567602,0.503788,0.190476,0.310345,0.671233,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1173,59ea397f,0.133333,1,0.550000,0.453751,0.293367,0.170455,0.285714,0.362069,0.410959,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2746,dd4b8b52,0.533333,1,0.583333,0.538686,0.382653,0.237374,0.523810,0.482759,0.438356,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2715,db49ac7c,0.466667,0,0.716667,0.350502,0.561224,0.285354,0.190476,0.482759,0.465753,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,a3bd8858,0.666667,1,0.750000,0.449168,0.798469,0.540404,0.142857,0.810345,0.739726,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2447,c4c2076b,0.133333,0,0.666667,0.687313,0.286990,0.267677,0.095238,0.568966,0.616438,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2630,d4a81b97,0.200000,1,0.433333,0.345059,0.484694,0.672980,0.190476,0.362069,0.726027,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1529,782a0589,0.333333,1,0.750000,0.322144,0.471939,0.212121,0.428571,0.431034,0.452055,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [62]:
dfTrain.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
count,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,...,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000,2859.000000
mean,0.331351,0.381952,0.497528,0.448753,0.472904,0.304847,0.341731,0.491618,0.504262,0.469405,...,0.239944,0.231549,0.236796,0.271773,0.247289,0.244141,0.230150,0.285414,0.217209,0.267226
std,0.214629,0.485950,0.189788,0.158781,0.169483,0.193630,0.207859,0.170828,0.176142,0.169014,...,0.427124,0.421896,0.425190,0.444952,0.431512,0.429652,0.421003,0.451691,0.412418,0.442589
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.133333,0.000000,0.400000,0.329252,0.346939,0.154040,0.190476,0.379310,0.383562,0.361111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.266667,0.000000,0.500000,0.408567,0.459184,0.255051,0.285714,0.482759,0.493151,0.444444,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.466667,1.000000,0.616667,0.530495,0.586735,0.429924,0.476190,0.603448,0.616438,0.569444,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [63]:
dfTest.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-Season_Summer,PCIAT-Season_Winter,SDS-Season_Fall,SDS-Season_Spring,SDS-Season_Summer,SDS-Season_Winter,PreInt_EduHx-Season_Fall,PreInt_EduHx-Season_Spring,PreInt_EduHx-Season_Summer,PreInt_EduHx-Season_Winter
count,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,...,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000,317.000000
mean,0.320294,0.328076,0.505678,0.459772,0.475689,0.310602,0.336263,0.487599,0.492330,0.469111,...,0.236593,0.201893,0.246057,0.312303,0.227129,0.214511,0.223975,0.337539,0.208202,0.230284
std,0.212542,0.470255,0.198574,0.161463,0.171834,0.184416,0.183568,0.173295,0.172413,0.176111,...,0.425662,0.402047,0.431393,0.464165,0.419639,0.411132,0.417564,0.473618,0.406664,0.421680
min,0.000000,0.000000,0.000000,0.164543,0.102041,0.012626,0.000000,0.017241,0.000000,0.013889,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,0.000000,0.416667,0.340697,0.338010,0.162879,0.190476,0.379310,0.383562,0.361111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.266667,0.000000,0.500000,0.428255,0.459184,0.273990,0.333333,0.465517,0.493151,0.458333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.466667,1.000000,0.666667,0.541479,0.599490,0.435606,0.428571,0.586207,0.602740,0.583333,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,0.966667,0.984836,0.924745,0.991162,0.904762,1.017241,1.000000,0.972222,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Export to csvs!

In [64]:
dfTrain.columns[dfTrain.columns.str.contains("PCIAT-PCIAT")]

Index(['PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
       'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',
       'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12',
       'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16',
       'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'PCIAT-PCIAT_Total'],
      dtype='object')

In [65]:
len(dfTrain.columns[dfTrain.columns.str.contains("PCIAT-PCIAT")])

21

In [66]:
dfTrain.shape

(2859, 107)

In [67]:
dfTrain = dfTrain.drop(columns=dfTrain.columns[dfTrain.columns.str.contains("PCIAT-PCIAT")])
dfTrain.shape

(2859, 86)

In [68]:
dfTest.shape

(317, 107)

In [69]:
dfTest = dfTest.drop(columns=dfTest.columns[dfTest.columns.str.contains("PCIAT-PCIAT")])
dfTest.shape

(317, 86)

In [70]:
dfTrain.to_csv('content/dataTrain.csv', index=False)
dfTest.to_csv('content/dataTest.csv', index=False)